Unzipping and adding the data to a pandas dataframe

In [2]:
import zipfile
import pandas as pd
z = zipfile.ZipFile("../data/vehicles.zip")
z.extractall()

vehicles = pd.read_csv("vehicles.csv")

In [3]:
vehicles.shape

(426880, 26)